In [239]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score, mean_squared_error;
from sklearn.model_selection import train_test_split,ShuffleSplit,GridSearchCV;
from fastFM.mcmc import FMClassification, FMRegression;
from sklearn.preprocessing import OneHotEncoder;
from fastFM import als;
from sklearn.feature_extraction.text import CountVectorizer;
import gc;
import pickle;
import random;
import matplotlib.pyplot as plt;
import time;

Doumentation : http://ibayer.github.io/fastFM/

### Model A: Basic model with just userId and movieId

In [599]:
def encode(df):
    
    '''
    Input : train and test sets
    Output : One hot encoded datasets
    '''
    
    encoder = OneHotEncoder(handle_unknown='ignore').fit(df)
    #trainX = encoder.transform(trainX)
    #testX = encoder.transform(testX)
    
    return encoder

In [107]:
ratings = np.genfromtxt('./ml-1m/ratings.dat',delimiter="::")
ratings =  pd.DataFrame(ratings)

ratings.columns = ['userId','movieId','rating','timestamp']
ratings = ratings.drop('timestamp', axis=1)
y = ratings['rating'].values
X = ratings.drop('rating', axis=1)

#trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.20, random_state=1234)
#trainX, testX = encode(trainX,testX)

encoder = encode(X)

##fixed
#trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.10, random_state=1234)
trainX = encoder.transform(trainX)
testX = encoder.transform(testX)

cv = ShuffleSplit(n_splits=3, test_size= int(0.2 * trainX.shape[0]), random_state = 2018)


estimator = als.FMRegression()

params = {
'n_iter' : np.arange(10,100,30),
'rank' :  np.arange(2,12,4),
'l2_reg_w': np.logspace(-6, -1, 3),
'l2_reg_V' : np.logspace(-6, -1, 3)
}

###Gridsearch over parameters
regressor = GridSearchCV(estimator=estimator , cv=cv, param_grid=params)
regressor.fit(trainX, trainY)

###get RMSE
mean_squared_error(regressor.predict(testX),testY)**0.5

0.8771354587448218

### Model B : Adding user data, genre, year of movie

In [114]:
users = pd.read_csv('./ml-1m/users.dat', sep='::', names=['userId', 'gender', 'age', 'occupation', 'zip'], \
                    header=None)
users.head()

/home/som/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


,userId,gender,age,occupation,zip
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


In [175]:
gc.collect()

2082

In [146]:
movies = pd.read_csv('./ml-1m/movies.dat', sep='::', names=['movieId', 'title', 'genres'], header=None)
movies.head()
movies['year'] = movies.title.apply(lambda x : x[-5:-1])

sparse_genres = pandas.DataFrame(CountVectorizer().fit_transform(movies.genres\
                                .map(lambda x: x.replace('|', ' '))).todense())

movies = pandas.concat([movies[['movieId']], sparse_genres], axis=1) 


ratings = np.genfromtxt('./ml-1m/ratings.dat',delimiter="::")
ratings =  pd.DataFrame(ratings)

ratings.columns = ['userId','movieId','rating','timestamp']
ratings = ratings.drop('timestamp', axis=1)

ratings = pandas.merge(pandas.merge(ratings, users, on='userId'), movies, on='movieId')


y = ratings['rating'].values
X = ratings.drop('rating', axis=1)

for feature in X.columns:
    _,X[feature] = numpy.unique(X[feature], return_inverse=True)

trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.20, random_state=1234)
encoder = encode(X)

##fixed
#trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.10, random_state=1234)
trainX = encoder.transform(trainX)
testX = encoder.transform(testX)



cv = ShuffleSplit(n_splits=3, test_size= int(0.2 * trainX.shape[0]), random_state = 2018)
estimator2 = als.FMRegression()

params = {
'n_iter' : np.arange(10,100,30),
'rank' :  np.arange(2,12,4),
'l2_reg_w': np.logspace(-6, -1, 3),
'l2_reg_V' : np.logspace(-6, -1, 3)
}

###Gridsearch over parameters
regressor2 = GridSearchCV(estimator=estimator2 , cv=cv, param_grid=params)
regressor2.fit(trainX, trainY)

###get RMSE
mean_squared_error(regressor2.predict(testX),testY)**0.5

0.8783405327095336

In [156]:
pickle.dump(regressor,open("./dump_regressor1","wb"))
pickle.dump(regressor2,open("./dump_regressor2","wb"))

In [158]:
regressor1 = pickle.load(open("./dump_regressor","rb"))

GridSearchCV(cv=ShuffleSplit(n_splits=3, random_state=2018, test_size=160033, train_size=None),
       error_score='raise',
       estimator=FMRegression(init_stdev=0.1, l2_reg=0, l2_reg_V=0.1, l2_reg_w=0.1, n_iter=100,
       random_state=123, rank=8),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_iter': array([10, 40, 70]), 'rank': array([ 2,  6, 10]), 'l2_reg_w': array([1.00000e-06, 3.16228e-04, 1.00000e-01]), 'l2_reg_V': array([1.00000e-06, 3.16228e-04, 1.00000e-01])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

```
Factorization Machines have been described as state of the art for many recommendation systems. 

Yet, experience has shown these models to suffer from slowtraining and local minima. Use a large(ish) dataset and characterize where FMs are easy to fit and accurate and where they are not.

1. Start with models that have no side information, and are only user and item ratings.
Specifically, subsample datasets from small to large, and subsample users/items
from sparsely-populated to well-populated, and train and test FMs. Where do they
work the best? Where do they fail? Can you set good rules of thumbs for their
training and use?
2. Next use side information about users or items. Answer the same questions as
above.

```

### Regressor 1 : No side information

In [621]:
def load_data_simple(file_):
    
    '''
    Input 
    
        File_ : file name of the ratings file
    
    Output : test - train dfs
    '''

    ratings = np.genfromtxt(file_,delimiter="::")
    ratings =  pd.DataFrame(ratings)
    
    ratings.columns = ['userId','movieId','rating','timestamp']
    ratings = ratings.drop('timestamp', axis=1)
    
    y = ratings['rating'].values
    X = ratings.drop('rating', axis=1)
    
    encoder = encode(X)
    

    
    ##fixed
    trainX, testX, trainY, testY = train_test_split(X, y, test_size=0.10, random_state=1234)
    trainX = encoder.transform(trainX)
    testX = encoder.transform(testX)
    
    return (X.userId.nunique(),X.movieId.nunique()),trainX, testX, trainY, testY

#### Subsample data from small to large

In [ ]:
#can grid search do this directly?

#train vs test error as data increase? vs a benchmark model, with and without early stopping vs SVD
#time taken to run the model
#Will try all k, epoch hyperspace together 

##FM without early stopping ?



# For the best iteration - check for local optima

#local optima - by sgd vs gd and changing random state?






In [661]:
def subsample_user_item(fname,sample_by_="All"):
    
    '''
    
    Input : trainX, testX, trainY, testY : dfs from the load_Data_Simple function
    
    sample_by_ : 
        All   -   small to large
        Items -   less to many
        Users -   less to many 
    
    '''
    cnt,trainX, testX, trainY, testY = load_data_simple(fname)  
    
    if sample_by_ == "All":
        #cv = ShuffleSplit(n_splits=10, test_size= int(0.2 * trainX.shape[0]), random_state = 2018)
        
        model_run = {}
        #keep lambda at
        _l = trainX.shape[0]
        
        ###10% for cv
        setcv = set(random.sample(range(_l), int(_l * .10)))
        potentialtrain = set(range(_l)) - setcv
        
        for i in np.arange(.2,1.01,0.2):
            #local split 
            settrain = random.sample(potentialtrain, int(len(potentialtrain) * i))
            print("model_" + sample_by_ +"_"+ str(i))
            model_run["model_" + sample_by_ +"_"+ str(i)] = fm_model(setcv,settrain,trainX,testX,trainY,testY)
        
        #return statistics
        return model_run

    if sample_by_ == "Items":
        # number of items in the dataset : 
        nusers = cnt[0]
        nitems = cnt[1]
        ###**modify**
        
        model_run = {}
        #keep lambda at
        _l = trainX.shape[0]
        
        ###10% for cv
        setcv = set(random.sample(range(_l), int(_l * .10)))
        potentialtrain = set(range(_l)) - setcv
        
        
        for i in [1,5,25,125,625,np.inf]:    #i defines atmost # of movies/items to be sampled
            #local split 
            settrain = set()
            #adds additional 1 min per model
            for j in range(nusers,nusers+nitems): #loop through movie columns
            #list(potentialtrain)# too much time to slice
                train_indices = np.argwhere(trainX[:,j])[:,0] #get non-zero indices which are legal 
                
                cand_indices = potentialtrain.intersection(set(train_indices))
                settrain.update(random.sample(cand_indices, min(i,len(cand_indices))))
                

            print("CV size",len(setcv)/(len(settrain)+len(setcv)))
            
            print("model_" + sample_by_ +"_"+ str(i))
            model_run["model_" + sample_by_ +"_"+ str(i)] = fm_model(setcv,settrain,trainX,testX,trainY,testY)        
        
        
        return model_run
    
    
    if sample_by_ == "Users":
        # number of items in the dataset : 
        nusers = cnt[0]
        #nitems = cnt[1]
        ###**modify**
        
        model_run = {}
        #keep lambda at
        _l = trainX.shape[0]
        
        ###10% for cv
        setcv = set(random.sample(range(_l), int(_l * .10)))
        potentialtrain = set(range(_l)) - setcv
        
        
        for i in [1,2,4,16,32,np.inf]:    #i defines atmost # of movies/items to be sampled
            #local split 
            settrain = set()
            #adds additional 1 min per model
            for j in range(nusers): #loop through movie columns
            #list(potentialtrain)# too much time to slice
                train_indices = np.argwhere(trainX[:,j])[:,0] #get non-zero indices which are legal 
                
                cand_indices = potentialtrain.intersection(set(train_indices))
                settrain.update(random.sample(cand_indices, min(i,len(cand_indices))))
                

            print("CV size",len(setcv)/(len(settrain)+len(setcv)))
            
            print("model_" + sample_by_ +"_"+ str(i))
            model_run["model_" + sample_by_ +"_"+ str(i)] = fm_model(setcv,settrain,trainX,testX,trainY,testY)        
        
        
        return model_run


In [663]:
def fm_model(setcv,settrain,trainX,testX,trainY,testY):

    '''
    
    return:
        dict of dict  to store size, time for run, train error, test error etc.
    '''
    
    ##Optimise with cv

    model_summary = {}
    _ = "_"
    
    
    ##Get baseline once
    #https://surprise.readthedocs.io/en/stable/
                #basic_algorithms.html#surprise.prediction_algorithms.baseline_only.BaselineOnly
    
    
    
    itercnt = 0
    ##24 models : hyperspace
    for n_iter in np.arange(50,100,25):
        for rank in  np.arange(2,8,2):
            for l2_reg in np.logspace(-4, 0, 4): 
                ##future plan : further granular level regularization
                    
                start = time.time()
                
                #als : coordinate descent
                estimator_x = als.FMRegression(n_iter = int(n_iter), rank = int(rank), l2_reg = l2_reg)
                estimator_x.fit(trainX[list(settrain)],trainY[list(settrain)])
                
                #estimator_x.fit(trainX[list(settrain)],trainY[list(settrain)])
                
                
                
                ###get RMSE
                train_rmse = mean_squared_error(estimator_x.predict(trainX[list(settrain)]),trainY[list(settrain)])**0.5
                cv_rmse =    mean_squared_error(estimator_x.predict(trainX[list(setcv)]),trainY[list(setcv)])**0.5
                
                #not being used for evaluation *********
                test_rmse = mean_squared_error(estimator_x.predict(testX),testY)**0.5
                
                #average rating of the train set
                baseline = np.mean(trainY[list(settrain)])
                train_baseline = mean_squared_error(np.repeat(baseline,len(testY)),testY)**0.5
                
                #Model summary to be returned for all model
                itercnt += 1
                model_ = {
                'train_rmse' : train_rmse,
                'test_rmse' : test_rmse,
                'cv_rmse' : cv_rmse,
                'train_baseline_rmse' : train_baseline,
                'model_obj' : estimator_x,
                'n_iter' : n_iter,
                'rank' :  rank, #k
                'l2_reg' : l2_reg,
                'train_size':len(settrain),
                #'l2_reg_V':l2_reg_V, #future
                #'l2_reg_w' : l2_reg_w,
                    
                'time' : time.time() - start
                    }
                
                print("\n Model " + str(itercnt) + "done." + "\t CV RMSE: " + str(cv_rmse))
                model_summary["model_seq_ "+str(itercnt) + _ + str(n_iter) +_+ str(rank) +_+ str(l2_reg)] = model_


    
    return model_summary

In [556]:
gc.collect()

1922

#### Subsample data from small to large

In [ ]:
###store function return
#trainX, testX, trainY, testY = load_data_simple('./ml-1m/ratings.dat')
basic_model = subsample_user_item('./ml-1m/ratings.dat')
#pickle object for later use

In [383]:
pickle.dump(basic_model,open("./dump_basic_model","wb"))

In [413]:
#check best RMSE
m = {}
mi = 10.0

for key in basic_model.keys():
    for k2 in basic_model[key].keys():
        if basic_model[key][k2]["cv_rmse"] <= mi:
            mi = basic_model[key][k2]["cv_rmse"]
            m[basic_model[key][k2]["cv_rmse"]] = key+"_"+k2
        
        
print("Best model: ",m[mi],"\t","best cv error",mi)     

Best model:  model_All_1.0_model_seq_ 12_50_6_1.0 	 best cv error 0.8668426911401514


#### Subsample user/items from sparsely populated to well populated

##### Items

In [ ]:
#subsample user/items
#Same performance graphs for low number of items to high, same for users.

##- where do they fail? where do they work best?
basic_model_sub_items = subsample_user_item('./ml-1m/ratings.dat','Items')
#basic_model_sub_users = subsample_user_item('./ml-1m/ratings.dat','Users')

In [666]:
pickle.dump(basic_model_sub_items,open("./dump_basic_model_sub_items","wb"))

In [668]:
#check best RMSE
m = {}
mi = 10.0

for key in basic_model_sub_items.keys():
    for k2 in basic_model_sub_items[key].keys():
        if basic_model_sub_items[key][k2]["cv_rmse"] <= mi:
            mi = basic_model_sub_items[key][k2]["cv_rmse"]
            m[basic_model_sub_items[key][k2]["cv_rmse"]] = key+"_"+k2
        
        
print("Best model: ",m[mi],"\t","best cv error",mi)     

Best model:  model_Items_inf_model_seq_ 12_50_6_1.0 	 best cv error 0.8603343366912477


##### Users

In [ ]:
basic_model_sub_users = subsample_user_item('./ml-1m/ratings.dat','Users')

In [667]:
pickle.dump(basic_model_sub_users,open("./dump_basic_model_sub_users","wb"))

In [669]:
#check best RMSE
m = {}
mi = 10.0

for key in basic_model_sub_users.keys():
    for k2 in basic_model_sub_users[key].keys():
        if basic_model_sub_users[key][k2]["cv_rmse"] <= mi:
            mi = basic_model_sub_users[key][k2]["cv_rmse"]
            m[basic_model_sub_users[key][k2]["cv_rmse"]] = key+"_"+k2
        
        
print("Best model: ",m[mi],"\t","best cv error",mi)     

Best model:  model_Users_inf_model_seq_ 20_75_4_1.0 	 best cv error 0.8651680026818478


### Regressor 2: Side information

#### Add feature 1 : performe the test1 and test2

#### Add feature 2 : performe the test1 and test2

#### repeat